In [332]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [333]:
#Combine test and train into one file
train['source']='train'
test['source']='test'
data = pd.concat([train, test],ignore_index=True)
print (train.shape, test.shape, data.shape)

(4990, 14) (3532, 13) (8522, 14)


In [334]:
data.isnull().sum()

Product_Fat_Content                  0
Product_Identifier                   0
Product_Price                        0
Product_Shelf_Visibility             0
Product_Supermarket_Identifier       0
Product_Supermarket_Sales         3532
Product_Type                         0
Product_Weight                    1463
Supermarket _Size                 2409
Supermarket_Identifier               0
Supermarket_Location_Type            0
Supermarket_Opening_Year             0
Supermarket_Type                     0
source                               0
dtype: int64

In [355]:
test.head()

,Product_Fat_Content,Product_Price,Product_Shelf_Visibility,Product_Type,Product_Weight,Supermarket _Size,Supermarket_Location_Type,Supermarket_Opening_Year,Supermarket_Type
4990,1,459.98,0.214125,1,8.270,2,3,2005,2
4991,1,464.98,0.127821,1,8.270,3,3,1994,1
4992,0,477.38,0.082171,1,7.390,3,3,1994,1
4993,1,472.63,0.011791,1,6.115,2,3,2005,2
4994,1,473.13,0.007038,1,6.115,3,3,1994,1


In [336]:
data['Product_Weight'] = data['Product_Weight'].fillna(12.9)

In [337]:
data['Supermarket _Size'] = data['Supermarket _Size'].fillna('Medium')

In [338]:
data['Product_Fat_Content'] = data['Product_Fat_Content'].replace({'Ultra Low fat':'Low Fat'})
print(data['Product_Fat_Content'].value_counts())

Low Fat       5516
Normal Fat    3006
Name: Product_Fat_Content, dtype: int64


In [339]:
data['Product_Type'].unique()

data['Product_Type'] = data['Product_Type'].replace({'Soft Drinks':1, 'Dairy':2, 'Hard Drinks':3, 'Canned':4, 'Frozen Foods':5,
       'Fruits and Vegetables':6, 'Snack Foods':7, 'Baking Goods':8,
       'Starchy Foods':9, 'Meat':10, 'Seafood':11, 'Breakfast':12, 'Breads':13,
       'Health and Hygiene':14, 'Household':15, 'Others':16})
print(data['Product_Type'].value_counts())

6     1232
7     1200
15     909
5      856
2      682
4      649
8      648
14     520
1      445
10     425
13     251
3      214
16     169
9      148
12     110
11      64
Name: Product_Type, dtype: int64


In [340]:
data['Supermarket _Size'].unique()

data['Supermarket _Size'] = data['Supermarket _Size'].replace({'High':3,'Medium':2,'Small':1})
print(data['Supermarket _Size'].value_counts())

2    5202
1    2388
3     932
Name: Supermarket _Size, dtype: int64


In [341]:
data['Supermarket_Location_Type'].unique()

data['Supermarket_Location_Type'] = data['Supermarket_Location_Type'].replace({'Cluster 3':3,'Cluster 2':2,'Cluster 1':1})
print(data['Supermarket_Location_Type'].value_counts())

3    3349
2    2785
1    2388
Name: Supermarket_Location_Type, dtype: int64


In [342]:
data['Product_Fat_Content'].unique()

data['Product_Fat_Content'] = data['Product_Fat_Content'].replace({'Low Fat':0,'Normal Fat':1})
print(data['Product_Fat_Content'].value_counts())

0    5516
1    3006
Name: Product_Fat_Content, dtype: int64


In [343]:
data['Supermarket_Type'].unique()

data['Supermarket_Type'] = data['Supermarket_Type'].replace({'Supermarket Type1':1,'Supermarket Type2':2,'Supermarket Type3':2,'Grocery Store':2})
print(data['Supermarket_Type'].value_counts())

1    5577
2    2945
Name: Supermarket_Type, dtype: int64


In [344]:
#Drop the columns which have been converted to different types:
data.drop(['Product_Identifier','Supermarket_Identifier',
           'Product_Supermarket_Identifier'],axis=1,inplace=True)

#Drop the columns which have been converted to different types:
#test.drop(['Product_Supermarket_Sales'],axis=1,inplace=True)

In [345]:
#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

In [346]:
#Drop the columns which have been converted to different types:
test.drop(['Product_Supermarket_Sales','source'],axis=1,inplace=True)

C:\Users\mo.yosiwealth\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [356]:
y = train['Product_Supermarket_Sales']
X = train[['Product_Weight','Product_Fat_Content','Product_Shelf_Visibility','Product_Price','Supermarket_Opening_Year',
           'Supermarket_Type','Supermarket _Size','Supermarket_Location_Type']]

Xtest = test[['Product_Weight','Product_Fat_Content','Product_Shelf_Visibility','Product_Price','Supermarket_Opening_Year',
           'Supermarket_Type','Supermarket _Size','Supermarket_Location_Type']]

In [357]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4990 entries, 0 to 4989
Data columns (total 8 columns):
Product_Weight               4990 non-null float64
Product_Fat_Content          4990 non-null int64
Product_Shelf_Visibility     4990 non-null float64
Product_Price                4990 non-null float64
Supermarket_Opening_Year     4990 non-null int64
Supermarket_Type             4990 non-null int64
Supermarket _Size            4990 non-null int64
Supermarket_Location_Type    4990 non-null int64
dtypes: float64(3), int64(5)
memory usage: 510.9 KB


In [358]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [359]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.5, learning_rate = 0.1,
                          max_depth = 5, alpha = 5, n_estimators = 15)

xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 3389.206240


In [360]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(normalize=True)

lr.fit(X_train, y_train)

lr_pred = lr.predict(X_test)

lr_accuracy = round(lr.score(X_train,y_train) * 100,2)
lr_accuracy

35.52

In [361]:
test_pred = lr.predict(Xtest)

In [364]:
sub = pd.read_csv("SampleSubmission.csv")

linear_submission = pd.DataFrame({'Product_Supermarket_Identifier':sub['Product_Supermarket_Identifier'],
                                  'Product_Supermarket_Sales': test_pred},
                                 columns=['Product_Supermarket_Identifier','Product_Supermarket_Sales'])

#to csv
linear_submission.to_csv('lin_reg.csv',index=False)